In [ ]:
import os
import sys
import time

#画图
import matplotlib as mpl 
import matplotlib.pyplot as plt
#在notebook里画图
%matplotlib inline
import numpy as np
import sklearn
#处理数据
import pandas as pd

import tensorflow as tf
from tensorflow import keras

# print(tf.__version__)
# print(sys.version_info)
# for module in mpl,np,pd,sklearn,tf,keras:
#     print(module.__name__,module.__version__)

In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
# print(housing.DESCR)
# print(housing.data.shape)
# print(housing.target.shape)

# import pprint
# pprint.pprint(housing.data[0:5])
# pprint.pprint(housing.target[0:5]) 


In [ ]:
from sklearn.model_selection import train_test_split
x_train_all,x_test,y_train_all,y_test = train_test_split(housing.data,housing.target,random_state =7)
x_train,x_valid,y_train,y_valid = train_test_split(x_train_all,y_train_all,random_state = 11)
# print(x_valid.shape,y_valid.shape)
# print(x_train.shape,y_train.shape)
# print(x_test.shape,y_test.shape)
# print(np.max(x_train),np.min(x_train))

In [ ]:
# x = (x-u均值)/ std方差
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [ ]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

In [ ]:
# RandomizedSearchCV
#1. 转化为sklearn model
def build_model(hidden_layers = 1,
               layer_size = 30,
               learning_rate = 3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(layer_size,activation = "relu",input_shape=x_train.shape[1:]))
    for _ in range(hidden_layers -1):
        model.add(keras.layers.Dense(layer_size,activation = 'relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss = 'mse',optimizer = optimizer)
    return model

sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)
    
callbacks = [
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-2)
]
history = sklearn_model.fit(x_train_scaled,y_train,epochs=100,validation_data=(x_valid_scaled,y_valid),callbacks=callbacks)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
#2. 定义参数集合
from scipy.stats import reciprocal
#f(x) = 1/(x*log(b/a)) a<= x <=b
# reciprocal.rvs(1e-4,1e-2,size=10)
param_distribution = {
    "hidden_layers":[1,2,3,4],
    "layer_size":np.arange(1,100)
    "learning_rate":reciprocal(1e-4,1e-2)
}
from sklearn.model_selection import RandomizedSearchCV
random_search_cv = RandomizedSearchCV(sklearn_model,
                                     param_distribution,
                                      cv = 3,#cross_validation 训练集分成n份 n-1训练 最后一份验证
                                     n_iter = 10,
                                     n_jobs =5)
#3. 搜索参数
random_search_cv.fit(x_train_scaled,y_train,epochs = 100,
                    validation_data = (x_valid_scaled,y_valid),callbacks = callbacks)
print(random_search_cv.best_params_)#最好的参数
print(random_search_cv.best_score_)#最小的loss
print(random_search_cv.best_estimator_)#最好的model

In [ ]:
model = random_serach_cv.best_estimator_.model
model.evaluate(x_test_scaled,y_test)